Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Aiman Lameesa"
ID = "st122876"

---

# Lab 06: Generative classifiers: Naive Bayes

As discussed in class, a Naive Bayes classifier works as follows:
$$\begin{eqnarray}
p(y \mid \mathbf{x} ; \theta) & = & \frac{p(\mathbf{x} \mid y ; \theta) p(y ; \theta)}{p(\mathbf{x} ; \theta)} \\
& \propto & p(\mathbf{x} \mid y ; \theta) p(y ; \theta) \\
& \approx & p(y ; \theta) \prod_j p(x_j \mid y ; \theta)
\end{eqnarray}$$
We will use Naive Bayes to perform diabetes diagnosis and text classification.

## Example 1: Diabetes classification

In this example we predict wheter a patient with specific diagnostic measurements has diabetes or not. As the features are
continuous, we will model the conditional probabilities
$p(x_j \mid y ; \theta)$ as univariate Gaussians with mean $\mu_{j,y}$ and standard deviation $\sigma_{j,y}$.

The data are originally from the U.S. National Institute of Diabetes and Digestive and Kidney Diseases (NIDDK) and are available
from [Kaggle](https://www.kaggle.com/uciml/pima-indians-diabetes-database)


In [2]:
import csv
import math
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

### Data manipulation

First we have some functions to read the dataset, split it into train and test, and partition it according to target class ($y$).

In [3]:
# Load data from CSV file
def loadCsv(filename):
    data_raw = pd.read_csv(filename)
    headers = data_raw.columns
    dataset = data_raw.values
    return dataset, headers

# Split dataset into test and train with given ratio
def splitDataset(test_size,*arrays,**kwargs):
    return train_test_split(*arrays,test_size=test_size,**kwargs)

# Separate training data according to target class
# Return key value pairs array in which keys are possible target variable values
# and values are the data records.

def data_split_byClass(dataset):
    Xy = {}
    for i in range(len(dataset)):
        datapair = dataset[i]
        # datapair[-1] (the last column) is the target class for this record.
        # Check if we already have this value as a key in the return array
        if (datapair[-1] not in Xy):
            # Add class as key
            Xy[datapair[-1]] = []
        # Append this record to array of records for this class key
        Xy[datapair[-1]].append(datapair)
    return Xy

### Model training

Next we have some functions used for training the model. Parameters include mean and standard deviation, used
to partition numerical variables into categorical variables, as well as 

In [4]:
# Parameters of a Gaussian are its mean and standard deviation

def mean(numbers):
    return sum(numbers)/float(len(numbers))

def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

# Calculate Gaussian parameters mu and sigma for each attribute over a dataset

def get_gaussian_parameters(X,y):
    parameters = {}
    unique_y = np.unique(y)
    for uy in unique_y:
        mean = np.mean(X[y==uy],axis=0)
        std = np.std(X[y==uy],axis=0)
        py = y[y==uy].size/y.size
        parameters[uy] = {'prior':py,'mean':mean,'std':std}
    return parameters, unique_y

def calculateProbability(x, mu, sigma):
    sigma = np.diag(sigma**2)
    x = x.reshape(-1,1)
    mu = mu.reshape(-1,1)
    exponent = np.exp(-1/2*(x-mu).T@np.linalg.inv(sigma)@(x-mu))
    return ((1/(np.sqrt(((2*np.pi)**x.size)*np.linalg.det(sigma))))*exponent)[0,0]

### Model testing

Next some functions for testing the model on a test set and computing its accuracy. Note that we assume
$$ p(y \mid \mathbf{x} ; \theta) \propto p(\mathbf{x} \mid y ; \theta), $$
which means we assume that the priors $p(y)$ are equal for each possible value of $y$.

In [5]:
# Calculate class conditional probabilities for given input data vector

def predict_one(x,parameters,unique_y,prior = True):
    probabilities = []
    for key in parameters.keys():
        probabilities.append(calculateProbability(x,parameters[key]['mean'],parameters[key]['std'])*(parameters[key]['prior']**(float(prior))))
    probabilities = np.array(probabilities)
    return unique_y[np.argmax(probabilities)]

def getPredictions(X, parameters, unique_y,prior=True):
    predictions = []
    for i in range(X.shape[0]):
        predictions.append(predict_one(X[i],parameters,unique_y,prior))
    return np.array(predictions)

# Get accuracy for test set

def getAccuracy(y, y_pred):
    correct = len(y[y==y_pred])
    return correct/y.size

### Experiment

Here we load the diabetes dataset, split it into training and test data, train a Gaussian NB model, and test the model on the test set.

In [8]:
# Load dataset

filename = 'diabetes.csv'
dataset, headers = loadCsv(filename)
#print(headers)
#print(np.array(dataset)[0:5,:])

# Split into training and test

X_train,X_test,y_train,y_test = splitDataset(0.4,dataset[:,:-1],dataset[:,-1])
print("Total =",len(dataset),"Train =", len(X_train),"Test =",len(X_test))

# Train model

parameters, unique_y = get_gaussian_parameters(X_train,y_train)
prediction = getPredictions(X_test,parameters,unique_y)
print("Accuracy with Prior =",getAccuracy(y_test,prediction))

# Test model

prediction = getPredictions(X_test,parameters,unique_y,prior = False)
print("Accuracy without Prior =",getAccuracy(y_test,prediction))

Total = 768 Train = 460 Test = 308
Accuracy with Prior = 0.775974025974026
Accuracy without Prior = 0.7597402597402597


###  Exercise In lab / take home work (20 points)

Find out the proportion of the records in your dataset are positive vs. negative.  Can we conclude that $p(y=1) = p(y=0)$? If not, add
the priors $p(y=1)$ and $p(y=0)$ to your NB model. Does it improve the result?



In [12]:
# YOUR CODE HERE

py_0 = len(X_train[y_train==0])/len(y_train)
py_1 = len(X_train[y_train==1])/len(y_train)

print('p(y = 0) is:', np.round_(py_0, decimals = 2))
print('p(y = 1) is:', np.round_(py_1, decimals = 2))

# raise NotImplementedError()

p(y = 0) is: 0.64
p(y = 1) is: 0.36


**Explain that you can conclude that $p(y=1) = p(y=0)$? If not, add
the priors $p(y=1)$ and $p(y=0)$ to your NB model. Does it improve the result? (double click to explain)**

Answer: It cannot be concluded that $p(y=1) = p(y=0)$ because $p(y=0)$ is 0.64 and $p(y=1)$ is 0.36. So, there is more samples of class 0 than class 1 in the dataset. 

After adding the priors, we get a slightly high accuracy since the priors consider the probability distribution of the classes over the entire dataset.

In [14]:
# Split into training and test

X_train,X_test,y_train,y_test = splitDataset(0.4,dataset[:,:-1],dataset[:,-1])
print("Total =",len(dataset),"Train =", len(X_train),"Test =",len(X_test))

# Train model

parameters, unique_y = get_gaussian_parameters(X_train,y_train)
prediction = getPredictions(X_test,parameters,unique_y, prior = True)
print("Accuracy with Prior =",getAccuracy(y_test,prediction))

# Test model

prediction = getPredictions(X_test,parameters,unique_y,prior = False)
print("Accuracy without Prior =",getAccuracy(y_test,prediction))

Total = 768 Train = 460 Test = 308
Accuracy with Prior = 0.7532467532467533
Accuracy without Prior = 0.7337662337662337


## Example 2: Text classification

This example has been adapted from a post by Jaya Aiyappan, available at
[Analytics Vidhya](https://medium.com/analytics-vidhya/naive-bayes-classifier-for-text-classification-556fabaf252b#:~:text=The%20Naive%20Bayes%20classifier%20is,time%20and%20less%20training%20data).

We will generate a small dataset of sentences that are classified as either "statements" or "questions."

We will assume that occurance and placement of words within a sentence is independent of each other
(i.e., the features are conditionally independent given $y$). So the sentence "this is my book" is the same as "is this my book."
We will treat words as case insensitive.

In [15]:
# Generate text data for two classes, "statement" and "question"

text_train = [['This is my novel book', 'statement'],
              ['this book has more than one author', 'statement'],
              ['is this my book', 'question'],
              ['They are novels', 'statement'],
              ['have you read this book', 'question'],
              ['who is the novels author', 'question'],
              ['what are the characters', 'question'],
              ['This is how I bought the book', 'statement'],
              ['I like fictional characters', 'statement'],
              ['what is your favorite book', 'question']]

text_test = [['this is the book', 'statement'], 
             ['who are the novels characters', 'question'], 
             ['is this the author', 'question'],
            ['I like apples']]

# Load training and test data into pandas data frames

training_data = pd.DataFrame(text_train, columns= ['sentence', 'class'])
print(training_data)
print('\n------------------------------------------\n')
testing_data = pd.DataFrame(text_test, columns= ['sentence', 'class'])
print(testing_data)


                             sentence      class
0               This is my novel book  statement
1  this book has more than one author  statement
2                     is this my book   question
3                     They are novels  statement
4             have you read this book   question
5            who is the novels author   question
6             what are the characters   question
7       This is how I bought the book  statement
8         I like fictional characters  statement
9          what is your favorite book   question

------------------------------------------

                        sentence      class
0               this is the book  statement
1  who are the novels characters   question
2             is this the author   question
3                  I like apples       None


In [16]:
# Partition training data by class

stmt_docs = [train['sentence'] for index,train in training_data.iterrows() if train['class'] == 'statement']
question_docs = [train['sentence'] for index,train in training_data.iterrows() if train['class'] == 'question']
all_docs = [train['sentence'] for index,train in training_data.iterrows()]

# Get word frequencies for each sentence and class

def get_words(text):
    # Initialize word list
    words = [];
    # Loop through each sentence in input array
    for text_row in text:       
        # Check the number of words. Assume each word is separated by a blank space
        # so that the number of words is the number of blank spaces + 1
        number_of_spaces = text_row.count(' ')
        # loop through the sentence and get words between blank spaces.
        for i in range(number_of_spaces):
            # Check for for last word
            words.append([text_row[:text_row.index(' ')].lower()])
            text_row = text_row[text_row.index(' ')+1:]  
            i = i + 1        
        words.append([text_row])
    return np.unique(words)

# Get frequency of each word in each document

def get_doc_word_frequency(words, text):  
    word_freq_table = np.zeros((len(text),len(words)), dtype=int)
    i = 0
    for text_row in text:
        # Insert extra space between each pair of words to prevent
        # partial match of words
        text_row_temp = ''
        for idx, val in enumerate(text_row):
            if val == ' ':
                 text_row_temp = text_row_temp + '  '
            else:
                  text_row_temp = text_row_temp + val.lower()
        text_row = ' ' + text_row_temp + ' '
        j = 0
        for word in words: 
            word = ' ' + word + ' '
            freq = text_row.count(word)
            word_freq_table[i,j] = freq
            j = j + 1
        i = i + 1
    
    return word_freq_table

In [17]:
# Get word frequencies for statement documents

word_list_s = get_words(stmt_docs)
word_freq_table_s = get_doc_word_frequency(word_list_s, stmt_docs)
tdm_s = pd.DataFrame(word_freq_table_s, columns=word_list_s)
print(tdm_s)

   are  author  book  bought  characters  fictional  has  how  i  is  like  \
0    0       0     1       0           0          0    0    0  0   1     0   
1    0       1     1       0           0          0    1    0  0   0     0   
2    1       0     0       0           0          0    0    0  0   0     0   
3    0       0     1       1           0          0    0    1  1   1     0   
4    0       0     0       0           1          1    0    0  1   0     1   

   more  my  novel  novels  one  than  the  they  this  
0     0   1      1       0    0     0    0     0     1  
1     1   0      0       0    1     1    0     0     1  
2     0   0      0       1    0     0    0     1     0  
3     0   0      0       0    0     0    1     0     1  
4     0   0      0       0    0     0    0     0     0  


In [18]:
# Get word frequencies over all statement documents

freq_list_s = word_freq_table_s.sum(axis=0) 
freq_s = dict(zip(word_list_s,freq_list_s))
print(freq_s)

{'are': 1, 'author': 1, 'book': 3, 'bought': 1, 'characters': 1, 'fictional': 1, 'has': 1, 'how': 1, 'i': 2, 'is': 2, 'like': 1, 'more': 1, 'my': 1, 'novel': 1, 'novels': 1, 'one': 1, 'than': 1, 'the': 1, 'they': 1, 'this': 3}


In [19]:
# Get word frequencies for question documents

word_list_q = get_words(question_docs)
word_freq_table_q = get_doc_word_frequency(word_list_q, question_docs)
tdm_q = pd.DataFrame(word_freq_table_q, columns=word_list_q)
print(tdm_q)

   are  author  book  characters  favorite  have  is  my  novels  read  the  \
0    0       0     1           0         0     0   1   1       0     0    0   
1    0       0     1           0         0     1   0   0       0     1    0   
2    0       1     0           0         0     0   1   0       1     0    1   
3    1       0     0           1         0     0   0   0       0     0    1   
4    0       0     1           0         1     0   1   0       0     0    0   

   this  what  who  you  your  
0     1     0    0    0     0  
1     1     0    0    1     0  
2     0     0    1    0     0  
3     0     1    0    0     0  
4     0     1    0    0     1  


In [20]:
# Get word frequencies over all question documents

freq_list_q = word_freq_table_q.sum(axis=0) 
freq_q = dict(zip(word_list_q,freq_list_q))
print(freq_q)
print(freq_list_s)
print(freq_list_q)

{'are': 1, 'author': 1, 'book': 3, 'characters': 1, 'favorite': 1, 'have': 1, 'is': 3, 'my': 1, 'novels': 1, 'read': 1, 'the': 2, 'this': 2, 'what': 2, 'who': 1, 'you': 1, 'your': 1}
[1 1 3 1 1 1 1 1 2 2 1 1 1 1 1 1 1 1 1 3]
[1 1 3 1 1 1 3 1 1 1 2 2 2 1 1 1]


In [21]:
# Get word probabilities for statement class
a = 1
prob_s = []
for count in freq_list_s:
    #print(word, count)
    prob_s.append((count+a)/(sum(freq_list_s)+len(freq_list_s)*a))
prob_s.append(a/(sum(freq_list_s)+len(freq_list_s)*a))
    
# Get word probabilities for question class

prob_q = []
for count in freq_list_q:
    prob_q.append((count+a)/(sum(freq_list_q)+len(freq_list_q)*a))
prob_q.append(a/(sum(freq_list_q)+len(freq_list_q)*a))   
    
    
print('Probability of words for "statement" class \n')
print(dict(zip(word_list_s, prob_s)))
print('------------------------------------------- \n')
print('Probability of words for "question" class \n')
print(dict(zip(word_list_q, prob_q)))

Probability of words for "statement" class 

{'are': 0.043478260869565216, 'author': 0.043478260869565216, 'book': 0.08695652173913043, 'bought': 0.043478260869565216, 'characters': 0.043478260869565216, 'fictional': 0.043478260869565216, 'has': 0.043478260869565216, 'how': 0.043478260869565216, 'i': 0.06521739130434782, 'is': 0.06521739130434782, 'like': 0.043478260869565216, 'more': 0.043478260869565216, 'my': 0.043478260869565216, 'novel': 0.043478260869565216, 'novels': 0.043478260869565216, 'one': 0.043478260869565216, 'than': 0.043478260869565216, 'the': 0.043478260869565216, 'they': 0.043478260869565216, 'this': 0.08695652173913043}
------------------------------------------- 

Probability of words for "question" class 

{'are': 0.05128205128205128, 'author': 0.05128205128205128, 'book': 0.10256410256410256, 'characters': 0.05128205128205128, 'favorite': 0.05128205128205128, 'have': 0.05128205128205128, 'is': 0.10256410256410256, 'my': 0.05128205128205128, 'novels': 0.0512820512

In [22]:
# Calculate prior for one class

def prior(className):    
    denominator = len(stmt_docs) + len(question_docs)
    
    if className == 'statement':
        numerator =  len(stmt_docs)
    else:
        numerator =  len(question_docs)
        
    return np.divide(numerator,denominator)
    
# Calculate class conditional probability for a sentence
    
def classCondProb(sentence, className):
    words = get_words(sentence)
    prob = 1
    for word in words:
        if className == 'statement':
            idx = np.where(word_list_s == word)
            prob = prob * prob_s[np.array(idx)[0,0]]
        else:
            idx = np.where(word_list_q == word)
            prob = prob * prob_q[np.array(idx)[0,0]]   
    
    return prob

# Predict class of a sentence

def predict(sentence):
    prob_statement = classCondProb(sentence, 'statement') * prior('statement')
    prob_question = classCondProb(sentence, 'question') * prior('question')
    if  prob_statement > prob_question:
        return 'statement'
    else:
        return 'question'

### In-lab exercise: Laplace smoothing

Run the code below and figure out why it fails.

When a word does not appear with a specific class in the training data, its class-conditional probability is 0, and we are unable to
get a reasonable probability for that class.

Research Laplace smoothing, and modify the code above to implement Laplace smoothing (setting the frequency of all words with frequency 0 to a frequency of 1).
Run the modified code on the test set.

In [23]:
test_docs = list([test['sentence'] for index,test in testing_data.iterrows()])
print('Getting prediction for "%s"' % test_docs[0])
predict(test_docs[0])


Getting prediction for "this is the book"


IndexError: index 0 is out of bounds for axis 1 with size 0

### Exercise 1.1 (10 points)

Explain Why it failed and explain how to solve the problem.

Explanation here! (Double click to explain)

It failed to execute when the word count is zero. If a word does not appear with a specific class in the training data, its class-conditional probability becomes 0 and thus we are unable to get a reasonable probability for that class.

To solve this problem, we can use Laplace smoothing. Laplace smoothing is a technique that helps to solve this problem of zero probability in the Naïve Bayes algorithm by changing the frequency of the words that have frequency 0 to frequency 1. Thus, we will always get a non-zero probability instead of a zero probability.

### Exercise 1.2 (20 points)

Modify your code and make it works.

In [24]:
# YOUR CODE HERE

# Calculate prior for one class

def prior(className):    
    denominator = len(stmt_docs) + len(question_docs)
    
    if className == 'statement':
        numerator =  len(stmt_docs)
    else:
        numerator =  len(question_docs)
        
    return np.divide(numerator,denominator)
    
# Calculate class conditional probability for a sentence
    
def classCondProb(sentence, className):
    words = get_words(sentence)
    prob = 1
    for word in words:
        if className == 'statement':
            
            idx = np.where(word_list_s == word)
            idx = np.array(idx)
            
            if idx.shape[1] != 0: 
                prob = (prob * prob_s[idx[0,0]]) 
            else:
                prob = (prob * prob_s[-1])
            
        else:
            
            idx = np.where(word_list_q == word)
            idx = np.array(idx)
            
            if idx.shape[1] != 0: 
                prob = (prob * prob_q[idx[0,0]]) 
            else:
                prob = (prob * prob_q[-1])  
    
    return prob

# Predict class of a sentence

def predict(sentence):
    prob_statement = classCondProb(sentence, 'statement') * prior('statement')
    prob_question = classCondProb(sentence, 'question') * prior('question')
    if  prob_statement > prob_question:
        return 'statement'
    else:
        return 'question'
    
# raise NotImplementedError()

In [25]:
# Get word probabilities for statement class
a = 1
prob_s = []
for count in freq_list_s:
    #print(word, count)
    prob_s.append((count+a)/(sum(freq_list_s)+len(freq_list_s)*a))
prob_s.append(a/(sum(freq_list_s)+len(freq_list_s)*a))
    
# Get word probabilities for question class

prob_q = []
for count in freq_list_q:
    prob_q.append((count+a)/(sum(freq_list_q)+len(freq_list_q)*a))
prob_q.append(a/(sum(freq_list_q)+len(freq_list_q)*a))   
    
    
print('Probability of words for "statement" class \n')
print(dict(zip(word_list_s, prob_s)))
print('------------------------------------------- \n')
print('Probability of words for "question" class \n')
print(dict(zip(word_list_q, prob_q)))

Probability of words for "statement" class 

{'are': 0.043478260869565216, 'author': 0.043478260869565216, 'book': 0.08695652173913043, 'bought': 0.043478260869565216, 'characters': 0.043478260869565216, 'fictional': 0.043478260869565216, 'has': 0.043478260869565216, 'how': 0.043478260869565216, 'i': 0.06521739130434782, 'is': 0.06521739130434782, 'like': 0.043478260869565216, 'more': 0.043478260869565216, 'my': 0.043478260869565216, 'novel': 0.043478260869565216, 'novels': 0.043478260869565216, 'one': 0.043478260869565216, 'than': 0.043478260869565216, 'the': 0.043478260869565216, 'they': 0.043478260869565216, 'this': 0.08695652173913043}
------------------------------------------- 

Probability of words for "question" class 

{'are': 0.05128205128205128, 'author': 0.05128205128205128, 'book': 0.10256410256410256, 'characters': 0.05128205128205128, 'favorite': 0.05128205128205128, 'have': 0.05128205128205128, 'is': 0.10256410256410256, 'my': 0.05128205128205128, 'novels': 0.0512820512

In [26]:
# Test function: Do not remove
test_docs = list([test['sentence'] for index,test in testing_data.iterrows()])

for sentence in test_docs:
    print('Getting prediction for %s"' % sentence)
    print(predict(sentence))
    
print("success!")
# End Test function

Getting prediction for this is the book"
question
Getting prediction for who are the novels characters"
question
Getting prediction for is this the author"
question
Getting prediction for I like apples"
question
success!


**Expect result**:\
Getting prediction for this is the book"\
question\
Getting prediction for who are the novels characters"\
question\
Getting prediction for is this the author"\
question

### Take home exercise

Find a more substantial text classification dataset, clean up the documents, and build your NB classifier. Write a brief report on your in-lab and take home exercises and results.

For take-home exercise, I will use a dataset of from the Kaggle. The dataset link is given below:

https://www.kaggle.com/datasets/shahrukhkhan/questions-vs-statementsclassificationdataset?select=train.csv

Since this training set is too large, I will use a subset of the dataset to build up a NB classifier.

In [28]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [29]:
import csv
import math
import re
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [30]:
# Load data from CSV file
def loadCsv(filename):
    data_raw = pd.read_csv(filename)
    headers = data_raw.columns
    dataset = data_raw.values
    return dataset, headers

# Split dataset into test and train with given ratio
def splitDataset(test_size,*arrays,**kwargs):
    return train_test_split(*arrays,test_size=test_size,**kwargs)

# Separate training data according to target class
# Return key value pairs array in which keys are possible target variable values
# and values are the data records.

def data_split_byClass(dataset):
    Xy = {}
    for i in range(len(dataset)):
        datapair = dataset[i]
        # datapair[-1] (the last column) is the target class for this record.
        # Check if we already have this value as a key in the return array
        if (datapair[-1] not in Xy):
            # Add class as key
            Xy[datapair[-1]] = []
        # Append this record to array of records for this class key
        Xy[datapair[-1]].append(datapair)
    return Xy

In [31]:
# loading the dataset

filename = 'train.csv'
dataset, headers = loadCsv(filename)

In [32]:
print(headers)
print(dataset[:5])

Index(['Unnamed: 0', 'doc', 'target'], dtype='object')
[[0
  ' For the rest of February, the Americans pushed north, and by 1 March, had taken two-thirds of the island.'
  0]
 [1
  'During the second half of the 19th century Dutch was banned from all levels of education by both Prussia and France and lost most of its functions as a cultural language. Other than France, what country does not give Dutch legal status as a language?'
  1]
 [2 'When did Egyptians start to make red pigment?' 1]
 [3
  " In a 2012 report, the National Center for State Courts' Court Statistics Project found that state trial courts received 103"
  0]
 [4 'What is an acceptable grounds for divorce for a Jehovah Witness' 1]]


In [33]:
dataset.shape

(126909, 3)

In [34]:
# taking first 1200 rows as the subset of the dataset

dataset = dataset[:1200,1:]

In [35]:
print(dataset[:5])

[[' For the rest of February, the Americans pushed north, and by 1 March, had taken two-thirds of the island.'
  0]
 ['During the second half of the 19th century Dutch was banned from all levels of education by both Prussia and France and lost most of its functions as a cultural language. Other than France, what country does not give Dutch legal status as a language?'
  1]
 ['When did Egyptians start to make red pigment?' 1]
 [" In a 2012 report, the National Center for State Courts' Court Statistics Project found that state trial courts received 103"
  0]
 ['What is an acceptable grounds for divorce for a Jehovah Witness' 1]]


In [36]:
# checking null values of the dataset

pd.DataFrame(dataset).isnull().sum()

0    0
1    0
dtype: int64

In [37]:
# removing special characters and converting uppercases to lowercases

for i in range(dataset.shape[0]):
    dataset[i,0] = re.sub(r"[^a-z]"," ",dataset[i,0].lower())

In [38]:
# spliting into training and test

X_train,X_test,y_train,y_test = splitDataset(0.3,dataset[:,:-1],dataset[:,-1])
print("Total =",len(dataset),
      "\nTrain =", len(X_train),
      "\nTest =",len(X_test))

Total = 1200 
Train = 840 
Test = 360


In [39]:
X_train[:10]

array([[' therefore  they are being reverse discriminated against because they are receiving the punishment that should be given to people who willingly and knowingly benefited from discriminatory practices'],
       ['the story sees bond pitted against the global criminal organisation spectre  marking the group s first appearance in an eon productions film since      s diamonds are forever  n    and tying craig s series of films together with an overarching storyline  which three personalities from previous films appear in spectre '],
       ['the      knesset elections marked a major turning point in israeli political history as menachem begin s likud party took control from the labor party '],
       ['   kilometres per hour     '],
       ['the first beer pump known in england is believed to have been invented by john lofting  b  in what country was john lofting born'],
       ['what was the bestselling drug in history '],
       ['the earthquake had a magnitude of    how long did 

In [40]:
# y_train[:10]

In [41]:
# Parameters of a Gaussian are its mean and standard deviation

def mean(numbers):
    return sum(numbers)/float(len(numbers))

def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

# Calculate Gaussian parameters mu and sigma for each attribute over a dataset

def get_gaussian_parameters(X,y):
    parameters = {}
    unique_y = np.unique(y)
    for uy in unique_y:
        mean = np.mean(X[y==uy],axis=0)
        std = np.std(X[y==uy],axis=0)
        py = y[y==uy].size/y.size
        parameters[uy] = {'prior':py,'mean':mean,'std':std}
    return parameters, unique_y

def calculateProbability(x, mu, sigma):
    sigma = np.diag(sigma**2)
    x = x.reshape(-1,1)
    mu = mu.reshape(-1,1)
    exponent = np.exp(-1/2*(x-mu).T@np.linalg.inv(sigma)@(x-mu))
    return ((1/(np.sqrt(((2*np.pi)**x.size)*np.linalg.det(sigma))))*exponent)[0,0]

In [42]:
# Calculate class conditional probabilities for given input data vector

def predict_one(x,parameters,unique_y,prior = True):
    probabilities = []
    for key in parameters.keys():
        probabilities.append(calculateProbability(x,parameters[key]['mean'],parameters[key]['std'])*(parameters[key]['prior']**(float(prior))))
    probabilities = np.array(probabilities)
    return unique_y[np.argmax(probabilities)]

def getPredictions(X, parameters, unique_y,prior=True):
    predictions = []
    for i in range(X.shape[0]):
        predictions.append(predict_one(X[i],parameters,unique_y,prior))
    return np.array(predictions)

# Get accuracy for test set

def getAccuracy(y, y_pred):
    correct = len(y[y==y_pred])
    return correct/y.size

In [43]:
# partitioning training data by class

stmt_docs = (X_train[y_train == 0]).reshape(-1).tolist()
ques_docs = (X_train[y_train == 1]).reshape(-1).tolist()

In [44]:
# Get word frequencies for each sentence and class

def get_words(text):
    # Initialize word list
    words = [];
    # Loop through each sentence in input array
    for text_row in text:       
        # Check the number of words. Assume each word is separated by a blank space
        # so that the number of words is the number of blank spaces + 1
        number_of_spaces = text_row.count(' ')
        # loop through the sentence and get words between blank spaces.
        for i in range(number_of_spaces):
            # Check for for last word
            words.append([text_row[:text_row.index(' ')].lower()])
            text_row = text_row[text_row.index(' ')+1:]  
            i = i + 1        
        words.append([text_row])
    return np.unique(words)

# Get frequency of each word in each document

def get_doc_word_frequency(words, text):  
    word_freq_table = np.zeros((len(text),len(words)), dtype=int)
    i = 0
    for text_row in text:
        # Insert extra space between each pair of words to prevent
        # partial match of words
        text_row_temp = ''
        for idx, val in enumerate(text_row):
            if val == ' ':
                 text_row_temp = text_row_temp + '  '
            else:
                  text_row_temp = text_row_temp + val.lower()
        text_row = ' ' + text_row_temp + ' '
        j = 0
        for word in words: 
            word = ' ' + word + ' '
            freq = text_row.count(word)
            word_freq_table[i,j] = freq
            j = j + 1
        i = i + 1
    
    return word_freq_table

In [45]:
# obtaining word frequencies for statement documents

word_list_s = get_words(stmt_docs)
word_freq_table_s = get_doc_word_frequency(word_list_s, stmt_docs)
tdm_s = pd.DataFrame(word_freq_table_s, columns=word_list_s)
print(tdm_s)

         a  aaron  abandon  abdi  abdication  abercrombie  able  abn  \
0    28  0      0        0     0           0            0     0    0   
1    29  1      0        0     0           0            0     0    0   
2    10  0      0        0     0           0            0     0    0   
3     8  0      0        0     0           0            0     0    0   
4    28  0      0        0     0           0            0     0    0   
..   .. ..    ...      ...   ...         ...          ...   ...  ...   
315  17  0      0        0     0           0            0     0    0   
316   7  1      0        0     0           0            0     0    0   
317  15  0      0        0     0           0            0     0    0   
318  33  1      0        0     0           0            0     0    0   
319  49  2      0        0     0           0            0     0    0   

     abnormal  ...  your  yuan  yugoslav  zamok  zedong  zenk  zhao  znak  \
0           0  ...     0     0         0      0       0   

In [46]:
# obtaining word frequencies over all statement documents

freq_list_s = word_freq_table_s.sum(axis=0) 
freq_s = dict(zip(word_list_s,freq_list_s))

# removing spaces

freq_s.pop('') 
list(freq_s.items())[:10]

[('a', 196),
 ('aaron', 1),
 ('abandon', 1),
 ('abdi', 1),
 ('abdication', 1),
 ('abercrombie', 1),
 ('able', 1),
 ('abn', 1),
 ('abnormal', 1),
 ('about', 7)]

In [47]:
# obtaining word frequencies for question documents

word_list_q = get_words(ques_docs)
word_freq_table_q = get_doc_word_frequency(word_list_q, ques_docs)
tdm_q = pd.DataFrame(word_freq_table_q, columns=word_list_q)
print(tdm_q)

         a  abandon  abbey  ability  able  aboard  abound  about  above  ...  \
0    60  0        0      0        0     0       0       0      0      0  ...   
1    25  0        0      0        0     0       0       0      0      0  ...   
2     7  0        0      0        0     0       0       0      0      0  ...   
3    16  1        0      0        0     0       0       0      0      0  ...   
4    53  0        0      0        0     0       0       0      0      0  ...   
..   .. ..      ...    ...      ...   ...     ...     ...    ...    ...  ...   
515  11  0        0      0        0     0       0       0      0      0  ...   
516  13  0        0      0        0     0       0       0      0      0  ...   
517  61  1        0      0        0     0       0       0      0      0  ...   
518   7  0        0      0        0     0       0       0      0      0  ...   
519  42  1        0      0        0     0       0       0      0      0  ...   

     york  you  young  youth  youths  y

In [48]:
# obtaining word frequencies over all question documents

freq_list_q = word_freq_table_q.sum(axis=0) 
freq_q = dict(zip(word_list_q,freq_list_q))

# removing spaces

freq_q.pop('') 
list(freq_q.items())[:10]

[('a', 173),
 ('abandon', 1),
 ('abbey', 1),
 ('ability', 1),
 ('able', 4),
 ('aboard', 1),
 ('abound', 1),
 ('about', 10),
 ('above', 1),
 ('abrasive', 1)]

In [49]:
# YOUR CODE HERE
a = 1
prob_s = []
for count in freq_list_s:
    #print(word, count)
    prob_s.append((count+a)/(sum(freq_list_s)+len(freq_list_s)*a))
    
prob_s.append(a/(sum(freq_list_s)+len(freq_list_s)*a))
    
# Get word probabilities for question class

prob_q = []
for count in freq_list_q:
    prob_q.append((count+a)/(sum(freq_list_q)+len(freq_list_q)*a))
    
prob_q.append(a/(sum(freq_list_q)+len(freq_list_q)*a))   


def prior(className):    
    denominator = len(stmt_docs) + len(ques_docs)
    
    if className == 'statement':
        numerator =  len(stmt_docs)
    else:
        numerator =  len(ques_docs)
        
    return np.divide(numerator,denominator)

# Calculate class conditional probability for a sentence
    
def classCondProb(sentence, className):
    words = get_words(sentence)
    prob = 1
    for word in words:
            if className == 'statement':
                idx = np.where(word_list_s == word)
                
                if np.array(idx).shape[1] != 0: 
                    prob = (prob * prob_s[np.array(idx)[0,0]]) 
                else:
                    prob = (prob * prob_s[-1])
                    
            else: #if className is question
                idx = np.where(word_list_q == word)
                
                if np.array(idx).shape[1] != 0: 
                    prob = (prob * prob_q[np.array(idx)[0,0]]) 
                else:
                    prob = (prob * prob_q[-1])
                
    
    return prob

# Predict class of a sentence

def predict(sentence):
    prob_statement = classCondProb(sentence, 'statement') * prior('statement')
    prob_question = classCondProb(sentence, 'question') * prior('question')
    if  prob_statement > prob_question:
        return 0 #statement
    else:
        return 1 #question

In [51]:
# obtaining prediction for the test set

yhat = []
for i in X_test:
    yhat.append(predict(i))

print ('The accuracy of the NB classifier for the test set is:', np.round(getAccuracy(y_test,yhat),2))

The accuracy of the NB classifier for the test set is: 0.79


In [52]:
print('The prior for \'statement\' class is:', np.round(prior('statement'),2))
print('The prior for \'question\' class is:', np.round(prior('question'),2))

print ('The accuracy of the NB classifier for the test set is:', np.round(getAccuracy(y_test,yhat),2))

The prior for 'statement' class is: 0.38
The prior for 'question' class is: 0.62
The accuracy of the NB classifier for the test set is: 0.79


In-lab exercise, we used the'diabetes.csv' dataset where we predict whether a patient with specific diagnostic measurements has diabetes or not. The dataset was split into training set and test set, where we trained a Gaussian NB model and tested the model on the test set. Then we also adding the priors while making predictions, which slightly improves the result.

We also used a small dataset of sentences to classify them as either "statements" or "questions." But when a word does not appear in a specific class in the train set, we cannot get a reasonable probability for that class. That is why, we implemented Laplace smoothing later that deals with the zero probability problem by replacing the frequency of the words that have frequency 0 with frequency 1.

For the take-home exercise, we selected a subset from the 'train.csv' dataset, cleaned up the dataset and built a NB classifier. We obtained the prediction on the test set, where we obtained 79% accuracy. The classifier considers the frequency of the words from the text and performs better than both the priors for statements and questions.